In [1]:
# coding: utf-8
import xml.etree.ElementTree as ETree
import pandas as pd
import json 
import utm

In [3]:
# Read xml
nodeData = '/Users/xinyun/Documents/Stage4A/data/Network.xml'
prstree = ETree.parse(nodeData)
root = prstree.getroot()

### **Get 2D list of node** 

In [4]:
# Create two lists
items = []
node = []

# Iterate through the root node to get the child node 
for n in root.iter('node') :
    id_nb = n.attrib.get('id')
    x = n.attrib.get('x')
    y = n.attrib.get('y')
    
    items = [id_nb, x, y]   # Create a list for each row
    node.append(items)

In [4]:
len(node)

17435

In [5]:
xmin = float(min(node[r][1] for r in range(len(node))))
ymin = float(min(node[r][2] for r in range(len(node))))
xmax = float(max(node[r][1] for r in range(len(node))))
ymax = float(max(node[r][2] for r in range(len(node))))

In [6]:
print(xmin, ymin, xmax, ymax)

594329.0264295612 5764460.89353358 652661.608364352 5843471.6513260715


In [7]:
(X0,Y0) = utm.to_latlon(xmin,ymin,31,'U')
(X1,Y1) = utm.to_latlon(xmax,ymax,31,'U')
print(X0, Y0, X1, Y1)

52.022752036643354 4.374775247912083 52.719564351976736 5.2603226194780905


In [7]:
xmax - xmin

58332.581934790825

In [8]:
ymax - ymin

79010.75779249147

### Get first 3 lines of nodes 

In [17]:
# Create two lists
items = []
node = []

# Iterate through the root node to get the child node 
counter = 0
for n in root.iter('node') :
    counter+=1
    if counter <=1 :
        id_nb = n.attrib.get('id')
        x = n.attrib.get('x')
        y = n.attrib.get('y')

        items = [id_nb, x, y]   # Create a list for each row
        node.append(items)
    else :
        break
    

### Get  first 10 lines of nodes

In [6]:
# Create two lists
items = []
node = []

# Iterate through the root node to get the child node 
counter = 0
for n in root.iter('node') :
    counter+=1
    if counter <=10 :
        id_nb = n.attrib.get('id')
        x = n.attrib.get('x')
        y = n.attrib.get('y')

        items = [id_nb, x, y]   # Create a list for each row
        node.append(items)
    else :
        break
    

### Get first 3 lines of links

In [18]:
# Create two lists
items_ = []
link = []
car = []
c = []
# Iterate through the root node to get the child node
counter = 0
for l in root.iter('link') :
    counter+=1
    if counter <= 1 :
        id_nb = l.attrib.get('id')
        start = l.attrib.get('from')
        end = l.attrib.get('to')
        length = l.attrib.get('length')
        modes = l.attrib.get('modes')
        items_ = [id_nb, start, end, length, modes]
        # List contains all kinds of modes of transport (car, pt)
        link.append(items_)
        # List contains only cars(car and pt,car)
        if (modes == 'pt,car') | (modes == 'car') :
            c = [id_nb, start, end, length, 'CAR']
            car.append(c)
    else :
        break

### **Get 2D list of link** 

In [19]:
# Create two lists
items_ = []
link = []
car = []
c = []
# Iterate through the root node to get the child node
for l in root.iter('link') :
    id_nb = l.attrib.get('id')
    start = l.attrib.get('from')
    end = l.attrib.get('to')
    length = l.attrib.get('length')
    modes = l.attrib.get('modes')
    items_ = [id_nb, start, end, length, modes]
    # List contains all kinds of modes of transport (car, pt)
    link.append(items_)
    # List contains only cars(car and pt,car)
    if (modes == 'pt,car') | (modes == 'car') :
        c = [id_nb, start, end, length, 'CAR']
        car.append(c)      

In [20]:
len(link)

31177

### Test of first 10 lines of link

In [8]:
# Create two lists
items_ = []
link = []
car = []
c = []
# Iterate through the root node to get the child node
counter = 0
for l in root.iter('link') :
    counter+=1
    if counter <= 10 :
        id_nb = l.attrib.get('id')
        start = l.attrib.get('from')
        end = l.attrib.get('to')
        length = l.attrib.get('length')
        modes = l.attrib.get('modes')
        items_ = [id_nb, start, end, length, modes]
        # List contains all kinds of modes of transport (car, pt)
        link.append(items_)
        # List contains only cars(car and pt,car)
        if (modes == 'pt,car') | (modes == 'car') :
            c = [id_nb, start, end, length, 'CAR']
            car.append(c)
    else :
        break

### **Dictionary and Json file**

In [19]:
# Empty nested dictionary
Network = { 'ROADS': {'NODES': {}, 'STOPS': {}, 'SECTIONS': {}, 'ZONES': {} },
         'LAYERS': [{}], 'TRANSIT': [] }

In [20]:
# Initialisation the first position of 'LAYERS'
Network['LAYERS'][0] = {'ID' : 'CAR', 'TYPE' : 'mnms.graph.layers.CarLayer',  'VEH_TYPE': 'mnms.vehicles.veh_type.Car',
    'DEFAULT_SPEED': 13.8,  'SERVICES': [], 'NODES': [], 'LINKS': [], 'MAP_ROADDB': {'NODES' : {}, 'LINKS' : {}}}

In [21]:
# Initialisation 

In [22]:
for part,info in Network.items() :
    if part == 'ROADS' :
        for key in info :
            if key == 'NODES' :  # Nodes part
                for i in range(len(node)) :
                    id_node = node[i][0]
                    x = float(node[i][1])
                    y = float(node[i][2])
                    info[key][id_node] = {}
                    info[key][id_node]['id'] = id_node
                    info[key][id_node]['position'] = [x,y]
            if key == 'SECTIONS' : # Sections part
                for i in range(len(link)) :
                    id_section = link[i][0]
                    upstream = link[i][1]
                    downstream = link[i][2]
                    length = float(link[i][3])
                    zone = 'RES'
                    info[key][id_section] = {'id' : id_section, 'upstream' : upstream, 'downstream' : downstream, 'length' : length, 'zone' : zone}
               # Stops part ......
            if key == 'ZONES' : # Zones part
                info[key]['RES'] = {}
                info[key]['RES']['id'] = 'RES'
                info[key]['RES']['sections'] = []
                info[key]['RES']['contour'] = [[xmin, ymin],[xmin, ymax], [xmax, ymin], [xmax, ymax]]
                for i in range(len(link)) :
                    id_section = link[i][0]
                    info[key]['RES']['sections'].append(id_section)
    if part == 'LAYERS' :
        
        # Car layer (first {})
        for key,value in  Network['LAYERS'][0].items() :
            if key == 'NODES' :
                for i in range(len(node)) :
                    ID = node[i][0]
                    X = float(node[i][1])
                    Y = float(node[i][2])
                    value.append({'ID' : ID, 'X' : X, 'Y' : Y, 'LABEL' : 'CAR', 'EXCLUDE_MOVEMENTS' : {}})
            if key == 'LINKS' :
                for i in range(len(car)) :
                    ID = car[i][0]
                    upstream = car[i][1]
                    downstream = car[i][2]
                    length = float(car[i][3])
                    label = car[i][4]
                    value.append({'ID' : ID, 'UPSTREAM': upstream, 'DOWNSTREAM' : downstream, 'COSTS' : {'length' : length}, 'LABEL' : label})
            if key == 'MAP_ROADDB' :
                for k,v in Network['LAYERS'][0]['MAP_ROADDB'].items() :
                    # Key = 'NODES'
                    if k == 'NODES' :
                        for i in range(len(node)) :
                            ID_node = node[i][0]  # Get node ID
                            v[ID_node] = ID_node
                    # Key = 'LINKS'
                    if k == 'LINKS' :
                        for i in range(len(car)) :
                            ID_link = car[i][0] # Get link ID
                            v[ID_link] = [ID_link]
                            
  #  if part == 'TRANSIT' :
   #     for i in range(len(car)) :
    #        ID = car[i][0]
     #       upstream = car[i][1]
      #      downstream = car[i][2]
      #      length = float(car[i][3])
       #     label = car[i][4]
        #    info.append({'ID' : ID, 'UPSTREAM': upstream, 'DOWNSTREAM' : downstream, 'LENGTH' : length,  'COSTS' : {'length' : length}, 'LABEL' : label})
            
        

In [23]:
# Serializing json  
json_object = json.dumps(Network, indent = 4) 

In [14]:
#print(json_object)

In [15]:
# Writing to network.json
with open("network.json", "w") as outfile:
    outfile.write(json_object)

In [16]:
with open("test_network.json", "w") as outfile:
    outfile.write(json_object)

In [17]:
with open("new_network.json", "w") as outfile:
    outfile.write(json_object)

In [24]:
with open("demon_network.json", "w") as outfile:
    outfile.write(json_object)